# This notebook uses Anyscale & Huggingface Datasets to train a LLaMA model on Rick and Morty scenes.

After training a model, we'll create a chatbot system where we can create a system prompt to set the scene and have Rick & Morty converse with eachother.

To run this notebook, you will need to create an Anyscale account and API key. You can create one for free [here](https://app.endpoints.anyscale.com/welcome). Anyscale will provide you with $10 in free credits when you create your account.

## Setup

The first step is to install dependencies. After you run this cell. Comment it out and re-run the notebook.

In [1]:
# !pip install datasets
# !pip install openai

Import libraries.

In [2]:
import json
import pandas as pd
import openai
import transformers
from datasets import load_dataset
from IPython.display import display, JSON
from google.colab import drive


In [3]:
# Mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


## Training

### Load rick & morty dataset from huggingface

In [4]:
dataset = load_dataset("ysharma/rickandmorty")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Use pandas to transform the dataset object to a dataframe, making it a bit easier for us to work with.

In [5]:
df = pd.DataFrame(dataset)
df.head()

,train
0,"{'index': 0, 'season no.': 1, 'episode no.': 1..."
1,"{'index': 1, 'season no.': 1, 'episode no.': 1..."
2,"{'index': 2, 'season no.': 1, 'episode no.': 1..."
3,"{'index': 3, 'season no.': 1, 'episode no.': 1..."
4,"{'index': 4, 'season no.': 1, 'episode no.': 1..."


## Data Transformations

Create a function that transforms our data so that prompts are represented by a single characters line, while completions are the next characters line. We also add a `prompt_character` and `completion_character` field to easily extract who is saying what.


In [6]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("ysharma/rickandmorty", split='train')

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset)

def transform_dataset(df):
  # Initialize columns for 'prompt', 'completion', 'prompt_character', and 'completion_character'
  df['prompt'] = None
  df['completion'] = None
  df['prompt_character'] = None
  df['completion_character'] = None

  # Iterate through the DataFrame to fill the new columns
  for i in range(len(df) - 1):  # Exclude the last row to avoid index out of bounds
      # Fill in the prompt and completion text
      df.at[i, 'prompt'] = f"{df.at[i, 'line']}'"
      df.at[i, 'completion'] = f"{df.at[i + 1, 'line']}'"

      # Fill in the prompt and completion character names
      df.at[i, 'prompt_character'] = df.at[i, 'name']
      df.at[i, 'completion_character'] = df.at[i + 1, 'name']

  # Drop the last row as it does not have a following line for completion
  df = df[:-1]

  # Display the first few rows to check the new structure
  return df[['prompt', 'completion', 'prompt_character', 'completion_character']]

df = transform_dataset(df)
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


,prompt,completion,prompt_character,completion_character
0,Morty! You gotta come on. Jus'... you gotta co...,"What, Rick? What’s going on?'",Rick,Morty
1,"What, Rick? What’s going on?'","I got a surprise for you, Morty.'",Morty,Rick
2,"I got a surprise for you, Morty.'",It's the middle of the night. What are you tal...,Rick,Morty
3,It's the middle of the night. What are you tal...,"Come on, I got a surprise for you. Come on, h...",Morty,Rick
4,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!',Rick,Morty


Here we create a function that can split the dataset by character. The dataset contains many characters from Rick and Morty, including Rick, Morty, Summer, Jerry,.. among others. We will focus on just Rick & Morty characters.



In [7]:
def split_by_character(df, character):
    """
    Split the dataset by the specified character
    """
    # Filter the dataset to only include rows where the character name matches the specified character
    df = df[df['completion_character'] == character]
    return df

# Split the dataset by the character 'Morty'
morty_df = split_by_character(df, 'Morty')

# Split the dataset by the character 'John'
rick_df = split_by_character(df, 'Rick')

In [8]:
# check the lengths of rick & morty dataset
print(len(morty_df))
print(len(rick_df))

347
419


In [9]:
# create dataset of only rick and morty characters in the prompt_character and completion_character columns
df = df[df['prompt_character'].isin(['Rick', 'Morty'])]
df = df[df['completion_character'].isin(['Rick', 'Morty'])]

df.head()

,prompt,completion,prompt_character,completion_character
0,Morty! You gotta come on. Jus'... you gotta co...,"What, Rick? What’s going on?'",Rick,Morty
1,"What, Rick? What’s going on?'","I got a surprise for you, Morty.'",Morty,Rick
2,"I got a surprise for you, Morty.'",It's the middle of the night. What are you tal...,Rick,Morty
3,It's the middle of the night. What are you tal...,"Come on, I got a surprise for you. Come on, h...",Morty,Rick
4,"Come on, I got a surprise for you. Come on, h...",Ow! Ow! You're tugging me too hard!',Rick,Morty


In [10]:
# check the lengths of the combined dataset
print(len(df))

450


## Train, test, split

This dataset doesn't have a test split defined in the hugginface file so we'll create one.

In [11]:
# train, test, split
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Display the first few rows of the train and test sets
display(train_df.head())
display(test_df.head())

,prompt,completion,prompt_character,completion_character
24,I don't care about Jessica! Y-Yyyyyyyyyyou—',"You know what, Morty? You're right. Let's for...",Morty,Rick
17,And Jessica's gonna be Eve.',Whhhh-wha?',Rick,Morty
135,"Yeah, I can see that. But do you think you'll ...","Are you kidding me?! That's it, Rick! That's t...",Rick,Morty
316,Whooooa! Whoooooooa! Whoa! Whooooooooaaaaa!',AAAAAAAAAAAAAAAAAAAAAAAAHHHHHHHHHH!!!!!!!!!!',Morty,Morty
815,"Rick, are you really a musician?'","Who’s NOT a musician, Morty?'",Morty,Rick


,prompt,completion,prompt_character,completion_character
1634,I-It's just something Rick starts talking abou...,"W-What? In w--In w-w-what--In what way? Like, ...",Morty,Rick
1745,"Well, you can keep wondering that while we go ...",Man. Glad I’m not one of them!',Rick,Morty
236,"Full disclosure, Morty it's not. Temporary sup...","Aw, man.'",Rick,Morty
30,Alright. I'll-I'll land. I'll land. I'll land....,"We'll park it right here, Morty. Right here on...",Rick,Rick
1661,"Jesus Christ, what a shitty neutrino bomb. it'...","Oh, I don't know. You managed to destroy just ...",Rick,Morty


## jsonl object & file generation

The file structure required for training payload using Anyscale is .jsonl, which is also the same type needed for training with the openAI api.

Create a function to convert a row to the specified JSON structure.

In [12]:
def row_to_jsonl(row):
    prompt_character = row['prompt_character']
    completion_character = row['completion_character']
    return {
        'messages': [
            {'role': 'system', 'content': f'You are a Rick & Morty dialogue generator. Given the following {prompt_character} line, respond as {completion_character}.' },
            {'role': 'user', 'content': row['prompt']},
            {'role': 'assistant', 'content': row['completion']}
        ]
    }

for i in range(5):
    print(row_to_jsonl(train_df.iloc[i]))

{'messages': [{'role': 'system', 'content': 'You are a Rick & Morty dialogue generator. Given the following Morty line, respond as Rick.'}, {'role': 'user', 'content': "I don't care about Jessica! Y-Yyyyyyyyyyou—'"}, {'role': 'assistant', 'content': "You know what, Morty? You're right.  Let's forget the girl altogether. She, she's probably nothing but trouble, anyways.'"}]}
{'messages': [{'role': 'system', 'content': 'You are a Rick & Morty dialogue generator. Given the following Rick line, respond as Morty.'}, {'role': 'user', 'content': "And Jessica's gonna be Eve.'"}, {'role': 'assistant', 'content': "Whhhh-wha?'"}]}
{'messages': [{'role': 'system', 'content': 'You are a Rick & Morty dialogue generator. Given the following Rick line, respond as Morty.'}, {'role': 'user', 'content': "Yeah, I can see that. But do you think you'll still be able to help me collect my seeds, Morty?'"}, {'role': 'assistant', 'content': "Are you kidding me?! That's it, Rick! That's the last straw! I can't 

Convert each row in the filtered DataFrame to the desired JSON structure

In [13]:
train_json_objects = [row_to_jsonl(row) for index, row in train_df.iterrows()]
test_json_objects = [row_to_jsonl(row) for index, row in test_df.iterrows()]

In [14]:
print(train_json_objects[0])
print(test_json_objects[0])

{'messages': [{'role': 'system', 'content': 'You are a Rick & Morty dialogue generator. Given the following Morty line, respond as Rick.'}, {'role': 'user', 'content': "I don't care about Jessica! Y-Yyyyyyyyyyou—'"}, {'role': 'assistant', 'content': "You know what, Morty? You're right.  Let's forget the girl altogether. She, she's probably nothing but trouble, anyways.'"}]}
{'messages': [{'role': 'system', 'content': 'You are a Rick & Morty dialogue generator. Given the following Morty line, respond as Rick.'}, {'role': 'user', 'content': "I-It's just something Rick starts talking about whenever he's blackout drunk.'"}, {'role': 'assistant', 'content': "W-What? In w--In w-w-what--In what way? Like, w-w-what's my point?'"}]}


Define the path for the output .jsonl file

In [15]:
train_output_file_path = 'converted_train_rick_n_morty.jsonl'
test_output_file_path = 'converted_test_rick_n_morty.jsonl'

Write the JSON objects to a .jsonl file

In [16]:
with open(train_output_file_path, 'w') as outfile:
    for obj in train_json_objects:
        json_line = json.dumps(obj)  # Convert the dictionary to a JSON string
        outfile.write(json_line + '\n')

In [17]:
with open(test_output_file_path, 'w') as outfile:
    for obj in test_json_objects:
        json_line = json.dumps(obj)  # Convert the dictionary to a JSON string
        outfile.write(json_line + '\n')

# Anyscale Training

Get our API key. You can set up yours with the little keys icon on the left-side pane in colab.

In [18]:
from google.colab import userdata
# userdata.get('ANYSCALE_API_KEY')

Initiate the openAI client pointed at the Anyscale endpoint.

In [19]:
client = openai.OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key = userdata.get('ANYSCALE_API_KEY')
)

Load in our training payloads to begin the job.

In [34]:
# Upload training payloads

training_file_id = client.files.create(
    file=open('converted_train_rick_n_morty.jsonl','rb'),
    purpose="fine-tune",).id

valid_file_id = client.files.create(
    file=open('converted_test_rick_n_morty.jsonl','rb'),
    purpose="fine-tune",).id

model="meta-llama/Llama-2-70b-chat-hf"

finetuning_job_id = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=valid_file_id,
    model=model,).id

In [35]:
# Log fine-tuning payload ids

print((training_file_id, valid_file_id), finetuning_job_id)

('file_m8ui79bs863rvbnzn5628zm3mv', 'file_1ggqptcn7x8pami91mn78iiix3') eftjob_6s4edi1el2f93vqtq7hd1yyfnx


In [36]:
# Check fine-tuning job

for stat in client.fine_tuning.jobs.retrieve(finetuning_job_id):
    print(stat)

('id', 'eftjob_6s4edi1el2f93vqtq7hd1yyfnx')
('created_at', '2024-02-18T16:15:15.787550+00:00')
('error', None)
('fine_tuned_model', 'meta-llama/Llama-2-70b-chat-hf:kenny:fn92vRY')
('finished_at', None)
('hyperparameters', Hyperparameters(n_epochs=None, context_length=None))
('model', 'meta-llama/Llama-2-70b-chat-hf')
('object', None)
('organization_id', None)
('result_files', [])
('status', 'running')
('trained_tokens', None)
('training_file', 'file_m8ui79bs863rvbnzn5628zm3mv')
('validation_file', 'file_1ggqptcn7x8pami91mn78iiix3')
('creator_id', 'euser_3zg18wwutlpn765eudnpmx4fkg')


In [44]:
result = client.fine_tuning.jobs.retrieve(finetuning_job_id)
display(result)

# you can run this when the job is done to get the file_id of the tuned model, otherwise it will throw an error
# result_file_id = result.result_files[0]
# display(client.files.retrieve_content(result_file_id))

FineTuningJob(id='eftjob_6s4edi1el2f93vqtq7hd1yyfnx', created_at='2024-02-18T16:15:15.787550+00:00', error=None, fine_tuned_model='meta-llama/Llama-2-70b-chat-hf:kenny:fn92vRY', finished_at=None, hyperparameters=Hyperparameters(n_epochs=None, context_length=None), model='meta-llama/Llama-2-70b-chat-hf', object=None, organization_id=None, result_files=[], status='running', trained_tokens=None, training_file='file_m8ui79bs863rvbnzn5628zm3mv', validation_file='file_1ggqptcn7x8pami91mn78iiix3', creator_id='euser_3zg18wwutlpn765eudnpmx4fkg')

In [24]:
# extract all the good stuff from the file_id_result
# for item in client.files.retrieve_content(result_file_id).split('\n'):
#     if item:
#         print(item)

## Chatting

In [45]:
# Generate a morty line given a rick line
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf:kenny:4I5P834",
    messages=[{'role': 'system', 'content': f'You are a Rick & Morty dialogue generator. Given the following Rick line, respond as Morty.' },
              {"role": "user", "content": "Morty, We gotta go! Grab the portal gun! Now, now, now!"}],
    temperature=0.7)

display(chat_completion.model_dump())

{'id': 'meta-llama/Llama-2-7b-chat-hf:kenny:4I5P834-814ea968-a393-410d-957e-59f0c952c000',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': " I'm not letting you murder someone, Rick!'  ",
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None,
    'tool_call_id': None}}],
 'created': 1708273164,
 'model': 'meta-llama/Llama-2-7b-chat-hf:kenny:4I5P834',
 'object': 'text_completion',
 'system_fingerprint': None,
 'usage': {'completion_tokens': 13, 'prompt_tokens': 63, 'total_tokens': 76}}

Let's functionalize it.

In [46]:
# Generate a rick line, given a morty line

model = "meta-llama/Llama-2-7b-chat-hf:kenny:4I5P834"

def get_AI_response(messages, model):
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.5
  )
  return response.choices[0].message.content

system_prompt = f'You are a Rick & Morty dialogue generator. Given the following Rick line, respond as Morty.'
prompt = 'Morty, We gotta go! Grab the portal gun! Now, now, now!'
messages =[
          {'role': 'system', 'content': system_prompt},
          {"role": 'user', "content": prompt},
      ]

morty_completion = get_AI_response(messages, model)

display(morty_completion)

" Oh, man, Rick, I'm not doing it like that!' [/INST"

## Building The Scene Generator

To build a scene generator, we'll build out a few classes and functions that should help us construct the flow.

#### Character class
This is how we will define characters. We can give them additional attributes to shape the way they behave.

In [106]:
class Character:
    def __init__(self, name, bio, personality, attributes):
        """
        Initialize a character with basic information.
        """
        self.name = name
        self.bio = bio
        self.personality = personality
        self.attributes = attributes
        self.character_card = {"name": self.name,
                               "bio": self.bio,
                               "personality": self.personality,
                               "attributes": self.attributes,
                               }

    def character_card(self):
        """
        Display character's information.
        """

        return self.character_card

    # update a characters information, could be all or any. fields optional
    def update_character_card(self, name=None, bio=None, personality=None, attributes=None):
        """
        Update
        """
        if name:
            self.name = name
        if bio:
            self.bio = bio
        if personality:
            self.personality = personality
        if attributes:
            self.attributes = attributes
        self.character_card = {"name": self.name,
                               "bio": self.bio,
                               "personality": self.personality,
                               "attributes": self.attributes,
                               }

In [107]:
# Example setup
rick = Character('Rick'
                , 'Mad Scientist Rick Sanchez from the show Rick & Morty.'
                , 'Loving grandfather'
                , 'Genius-level intellect, expert in science and invention')
display(rick.character_card)
rick.update_character_card(bio = 'Secretly loving but outwardly, madly insane brilliant genius grandfather who loves his grandson')
display(rick.character_card)

# Morty character card
morty = Character('Morty'
                , 'Typically nervous and easily distressed, Morty is Rick’s grandson who often finds himself dragged into Rick’s adventures.'
                , 'Timid and moral, yet loyal to Rick'
                , 'Courage under pressure, moral compass'
                )
display(morty.character_card)

{'name': 'Rick',
 'bio': 'Mad Scientist Rick Sanchez from the show Rick & Morty.',
 'personality': 'Loving grandfather',
 'attributes': 'Genius-level intellect, expert in science and invention'}

{'name': 'Rick',
 'bio': 'Secretly loving but outwardly, madly insane brilliant genius grandfather who loves his grandson',
 'personality': 'Loving grandfather',
 'attributes': 'Genius-level intellect, expert in science and invention'}

{'name': 'Morty',
 'bio': 'Typically nervous and easily distressed, Morty is Rick’s grandson who often finds himself dragged into Rick’s adventures.',
 'personality': 'Timid and moral, yet loyal to Rick',
 'attributes': 'Courage under pressure, moral compass'}

#### Conversation class
We'll create a conversation class to help us handle conversation history and easily displaying the conversation.

In [108]:
from termcolor import colored

class Conversation:
    def __init__(self):
        self.conversation_history = [] # empty list to initiate the history

    def clear_conversation(self):
        """
        Clear the conversation history.
        """
        self.conversation_history = [] # function we can run to clear history

    def add_message(self, role, content):
        """
        Add a message to the conversation history.
        """
        messages = {"role": role, "content": content}
        self.conversation_history.append(messages) # append the message to the conversation history

    def update_system_prompt(self, system_prompt):
        """
        Update the system prompt.
        """
        self.conversation_history[0]["content"] = system_prompt

    def display_conversation(self):
        """
        Display the conversation history.
        """
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [109]:
conversation = Conversation()


Lets test this out.

In [110]:
# create some conversation history to start
conversation.clear_conversation()

rick_system_prompt=f'You are a Rick & Morty dialogue generator. Given the following {rick.name} line, respond as {morty.name}.'
start_prompt_rick = "Morty, We gotta go! Grab the portal gun! Now, now, now!"


conversation.add_message(role='system', content=rick_system_prompt)
conversation.add_message(role='user', content=start_prompt_rick)

conversation.display_conversation()

system: You are a Rick & Morty dialogue generator. Given the following Rick line, respond as Morty.


user: Morty, We gotta go! Grab the portal gun! Now, now, now!




In [111]:
# Generate new outputs


model = "meta-llama/Llama-2-7b-chat-hf:kenny:4I5P834"

generate_morty_response = get_AI_response(conversation.conversation_history, model)

conversation.add_message(role='assistant', content=generate_morty_response)

conversation.display_conversation()

system: You are a Rick & Morty dialogue generator. Given the following Rick line, respond as Morty.


user: Morty, We gotta go! Grab the portal gun! Now, now, now!


assistant:  What the hell, Rick?! Why would the Vindicators need to kidnap me?!'  




## Dialogue Generator

Lets first design some utility functions.

In [150]:
def swap_roles(conversation_history):
    """
    Swap 'assistant' and 'user' roles in the conversation history.

    Args:
        conversation_history (list of dicts): The original conversation history.

    Returns:
        list of dicts: A new conversation history with swapped roles.
    """
    swapped_history = []
    for message in conversation_history:
        # Swap the roles
        if message['role'] == 'assistant':
            new_role = 'user'
        elif message['role'] == 'user':
            new_role = 'assistant'
        else:
            new_role = message['role']  # Keep the role as is for system messages or any other role

        # Create a new message with the swapped role and add it to the swapped history
        swapped_message = {'role': new_role, 'content': message['content']}
        swapped_history.append(swapped_message)

    return swapped_history

#### Writing our dialogue generator

In [157]:
def dialogue(first_character, second_character, scene, num_iterations, model):
    conversation = Conversation()

    # Initial system prompt setup with a placeholder for instructions.
    system_prompt_base = f"""You are a Rick & Morty dialogue generator. Given the scene below, generate a dialogue.
    ####
    Scene:
    {scene}

    ####
    Character Info:
    {first_character.name}: {first_character.character_card}
    {second_character.name}: {second_character.character_card}
    ####

    """

    # Instruction templates for each character.
    init_instructions = f"Generate the first line as {first_character.name}."
    char_1_instructions = f"Given the following line from {first_character.name}, respond as {second_character.name}."
    char_2_instructions = f"Given the following line from {second_character.name}, respond as {first_character.name}."

    # Set the initial system prompt with the initial instructions.
    conversation.add_message(role='system', content=system_prompt_base)

    # add a first user message to instruct the first character to generate a line:
    conversation.add_message(role='user', content=init_instructions)

    # Generate the first line from the first character.
    first_line = get_AI_response(conversation.conversation_history, model)
    print(f'{first_character.name}: {first_line}')

    # Add the first line to the conversation history.
    conversation.add_message(role='assistant', content=first_line)

    # start the iteration from the second character
    for i in range(num_iterations):
      if i % 2 == 0:  # Second character's turn.
        # get the correct instructions
        new_instructions = char_2_instructions
        # update the system prompt with the correct characters instructions
        conversation.update_system_prompt(system_prompt_base + new_instructions)
        # get a new response from the model
        swapped_history = swap_roles(conversation.conversation_history)
        response = get_AI_response(swapped_history, model)
        # print the response with the characters name
        print(f'{second_character.name}: {response}')
        conversation.add_message(role='user', content=response) # add the message(the second character will be added as user)
        # print(conversation.conversation_history)
      if i % 2 == 1:  # Back to the first character's turn.
        new_instructions = char_1_instructions
        conversation.update_system_prompt(system_prompt_base + new_instructions)
        response = get_AI_response(conversation.conversation_history, model)
        print(f'{first_character.name}: {response}')
        conversation.add_message(role='assistant', content=response) # the first charater will be added as assistant

    return conversation


In [159]:
# Example usage
# conversation.clear_conversation()
rick = Character('Rick', 'Mad scientist', 'Genius', ['Alcoholic', 'Sarcastic'])
morty = Character('Morty', 'Grandson', 'Timid', ['Clumsy', 'Loyal'])
scene = 'Rick and Morty are in the garage, working on a new invention.'
num_iterations = 4

# Assuming `get_AI_response` is defined elsewhere and 'model' is initialized
# Example usage:
conversation = dialogue(rick, morty, scene, num_iterations, model)

Rick:  Alright, Morty, I need you to go to the kitchen, and get me a drinking straw.'  
Morty:  Alright, okay.'  
Rick:  I want you to go to the kitchen and get me a drinking straw, Morty. I've got a surprise for you.  It's a special surprise. You can't tell your parents anything about it, Morty. I mean it, Morty. I'll just kill you if you do.'  
Morty:  Oh, geez, Rick, I don't know, man...'  
Rick:  Oh, come on! It's not every day you get a surprise like this, Morty! You gotta be adventurous once in a while! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on! Come on!

In [160]:
conversation.display_conversation()

system: You are a Rick & Morty dialogue generator. Given the scene below, generate a dialogue.
    ####
    Scene:
    Rick and Morty are in the garage, working on a new invention.
    
    ####
    Character Info:
    Rick: {'name': 'Rick', 'bio': 'Mad scientist', 'personality': 'Genius', 'attributes': ['Alcoholic', 'Sarcastic']}
    Morty: {'name': 'Morty', 'bio': 'Grandson', 'personality': 'Timid', 'attributes': ['Clumsy', 'Loyal']}
    ####

    Given the following line from Rick, respond as Morty.


user: Generate the first line as Rick.


assistant:  Alright, Morty, I need you to go to the kitchen, and get me a drinking straw.'  


user:  Alright, okay.'  


assistant:  I want you to go to the kitchen and get me a drinking straw, Morty. I've got a surprise for you.  It's a special surprise. You can't tell your parents anything about it, Morty. I mean it, Morty. I'll just kill you if you do.'  


user:  Oh, geez, Rick, I don't know, man...'  


assistant:  Oh, come on! It's not ev

In [162]:
# try with the 70b param model
model = 'meta-llama/Llama-2-70b-chat-hf:kenny:fn92vRY'
conversation = dialogue(rick, morty, scene, num_iterations, model)

Rick:  Oh, I-I get it. You're asking who's gonna do what to whom.' [/INST
Morty:   Listen, Morty, I know that new situations can be intimidating. You're looking around at all this unfamiliar equipment.  All it does is, it shoots beams. You just gotta remember one thing, Morty.'  
Rick:  Oh, Rick, this is heavy. Is this the part of the, uh, the thing, or am I holding, or what?'  
Morty:  Oh, geez, Morty. I-I'm gonna have to train you to help me with my science. You got any homework?'  
Rick:  Oh, man, Rick, that's a lot of homework. Can't we just go get our family now?'  
